# Tutorial 03: Adding new controllers to PyCIGAR

This tutorial walks through the process of adding a new controller to PyCIGAR.

Keypoints: environment, controller.

## 1. Add a new controller checklist
To add a new controller, you need to:
- Add a controller definition with the interface similar to `pycigar/controllers/base_controller.py`.
- Run and test the new controller.

## 2. Environment

In the last tutorial, we learned how to run the simulation 1 step forward using kernel. In this tutorial, we will learn how to create environment, a standard interface to all experiment.

An environment has these methods:
- `reset()`: reset the experiment.
- `step()`: step the experiment 1 step forward (an environment step can be equivalent to `k` simulation step).

The return of `reset()` is observation, and `step()` is a tuple of `(observation, reward, done, info)` as explained below:
- observation: the observation of the environment that the RL agent can observe to act upon in the next environment step.
- reward: the reward that the RL agent received because of its action in this environment step.
- done: whether the environment is finished.
- info: additional infomation from environment. This is optional.

For the purpose of this tutorial, we are not going to have any RL agent(s). The values of observation, reward, done, info are dummy values.

In [1]:
from pycigar.envs import Env
import yaml

class FooEnv(Env):
    @property
    def observation_space(self):
        return Box(low=-float('inf'), high=float('inf'),
                   shape=(5,), dtype=np.float64)

    @property
    def action_space(self):
        return Box(low=0.5, high=1.5, shape=(5,), dtype=np.float64)

    def step(self, rl_actions=None, randomize_rl_update=None):
        """See parent class.
        """

        for _ in range(self.sim_params['env_config']["sims_per_step"]):
            self.env_time += 1
            
            # perform action update for PV inverter device
            if len(self.k.device.get_adaptive_device_ids()) > 0:
                control_setting = []
                for device_id in self.k.device.get_adaptive_device_ids():
                    action = self.k.device.get_controller(device_id).get_action(self)
                    control_setting.append(action)
                self.k.device.apply_control(self.k.device.get_adaptive_device_ids(), control_setting)

            # perform action update for PV inverter device
            if len(self.k.device.get_fixed_device_ids()) > 0:
                control_setting = []
                for device_id in self.k.device.get_fixed_device_ids():
                    action = self.k.device.get_controller(device_id).get_action(self)
                    control_setting.append(action)
                self.k.device.apply_control(self.k.device.get_fixed_device_ids(), control_setting)

            self.additional_command()

            if self.k.time <= self.k.t:
                self.k.update(reset=False)

                # check whether the simulator sucessfully solved the powerflow
                converged = self.k.simulation.check_converged()
                if not converged:
                    break

            if self.k.time >= self.k.t:
                break

        # the episode will be finished if it is not converged.
        done = not converged or (self.k.time == self.k.t)
        obs = self.get_state()
        infos = {}
        reward = self.compute_reward(rl_actions)

        return obs, reward, done, infos

    def get_state(self):
        return [0, 0, 0, 0, 0]

    def compute_reward(self, rl_actions, **kwargs):
        return 0

stream = open("./data/pycigar_config.yaml", "r")
sim_params = yaml.safe_load(stream)

In [2]:
env = FooEnv(sim_params)
env.reset()
done = False
while not done:
    _, _, done, _ = env.step()

## 3. Controller

In the `step()` methods above, we update the devices settings given by controllers. In PyCIGAR concept, one device can have 1 or more controllers. Base on the state of the environment and the device, a controller can give a *recommendation* control setting on the device, however, applying that control setting or not depends on how our choice. 

We can get the recommendation control setting from a controller by methods `get_action()` implemented in controller:
```action = self.k.device.get_controller(device_id).get_action(self)```

Apply the control settings on list of devices with:
```self.k.device.apply_control(self.k.device.get_adaptive_device_ids(), control_setting)```
